# Import libraries

Here I made the same experiment as with transformers, but using pytorch lightning. I just wanted to show another approach, and also I use pl as my standard framework. I also used vanilla ResNet instead of Vision Transformer.

In [ ]:
#!pip install pytorch-lightning

In [ ]:
#!pip install neptune-client

In [ ]:
#!pip install opencv-python

In [1]:
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision
import torchvision.models as models
import pytorch_lightning as pl
import torch.nn as nn
from torchmetrics import Accuracy
import torch.optim as optim
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from collections import Counter

# Data

In [3]:
CONFIG = {"lr":2*1e-4,\
         "batch_size":32,\
         'epochs':10}

In [4]:
train_transforms = transforms.Compose([transforms.Resize(size=224),
              transforms.RandomHorizontalFlip(),
              transforms.RandomVerticalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
val_transforms = transforms.Compose([
              transforms.Resize(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])


In [5]:
train_ds = ImageFolder('../dataset/PLD_3_Classes_256/Training/',transform=train_transforms)
val_ds = ImageFolder('../dataset/PLD_3_Classes_256/Validation/',transform=train_transforms)
test_ds = ImageFolder('../dataset/PLD_3_Classes_256/Testing/',transform=train_transforms)

Let's look at class balance

In [6]:
train_cnt = Counter(train_ds.targets)
val_cnt = Counter(val_ds.targets)
test_cnt = Counter(test_ds.targets)
print(train_cnt)
print(val_cnt)
print(test_cnt)

Counter({0: 1303, 2: 1132, 1: 816})
Counter({0: 163, 2: 151, 1: 102})
Counter({0: 162, 2: 141, 1: 102})


So all splits looks like pretty balanced.

In [7]:
train_dl = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=4)
val_dl = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=4)

# Model

In [10]:
class LeafModel(pl.LightningModule):
    def __init__(self, num_classes, lr=2e-4):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.num_classes = num_classes
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 128),
            nn.Dropout(0.3),
            nn.Linear(128, self.num_classes)
        )
        
        self.loss_fn = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=3)
        
    def forward(self, x):
        x = self.model(x)
        return x
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2)
        return [optimizer], [scheduler]
    
    def training_step(self, batch, batch_idx):
        
        x, y = batch
        
        preds = self(x)
        loss = self.loss_fn(preds, y)
        acc = self.accuracy(torch.argmax(preds, dim=1), y)
        
        self.log('train_loss', loss.item(), on_epoch=True,logger=True)
        self.log('train_acc', acc, on_epoch=True,logger=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        
        x,y = batch
        
        preds = self(x)
        
        loss = self.loss_fn(preds, y)
        acc = self.accuracy(torch.argmax(preds, dim=1), y)
        
        self.log('val_loss', loss, on_epoch=True,prog_bar=True,logger=True)
        self.log('val_acc', acc, on_epoch=True,prog_bar=True,logger=True)
        
    def test_step(self, batch, batch_idx):
        
        x,y = batch
        preds = self(x)
        acc = self.accuracy(torch.argmax(preds, dim=1), y)
        
        self.log('test_acc', acc, on_epoch=True,prog_bar=True)

# Train and eval

I used neptune.ai as logger, because it's one of my favorite loggers (of course there is popular wandb and standard tensorboard).

In [11]:
from pytorch_lightning.loggers import NeptuneLogger

neptune_logger = NeptuneLogger(
    project="markpotanin/leaf-classifier",
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhNzgyN2Q3OC05ZWE5LTRkZGQtODUyOS03ZmYzNGQwOTczYTEifQ==",
    tags=["training", "resnet50"]
)

checkpoint_callback = ModelCheckpoint(dirpath='./resnet-potato/',filename='best', monitor='val_loss',mode='min', save_top_k=1)
model = LeafModel(num_classes = 3,lr = CONFIG['lr'])
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.001, patience=3, verbose=False, mode="min")
trainer_args = {
        "accelerator": "gpu",
        "max_epochs": CONFIG['epochs'],
        "callbacks": [early_stop_callback,checkpoint_callback],
    }
trainer = pl.Trainer(**trainer_args,logger = neptune_logger,default_root_dir='./resnet-potato/',enable_progress_bar=True)
trainer.fit(model,train_dl,val_dl)

/srv/conda/envs/saturn/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/srv/conda/envs/saturn/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/srv/conda/envs/saturn/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/jovyan/workspace/resnet-potato exists and is not empty.
  r

https://app.neptune.ai/markpotanin/leaf-classifier/e/LEAF-8


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

File /home/jovyan/workspace/resnet-potato/best.ckpt changed during upload, restarting upload.


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

File /home/jovyan/workspace/resnet-potato/best.ckpt changed during upload, restarting upload.


`Trainer.fit` stopped: `max_epochs=10` reached.


In [12]:
metrics = trainer.logged_metrics

In [13]:
metrics

{'train_loss_step': tensor(0.0010),
 'train_acc_step': tensor(1.),
 'val_loss': tensor(0.0126),
 'val_acc': tensor(0.9952),
 'train_loss_epoch': tensor(0.0110),
 'train_acc_epoch': tensor(0.9978)}

In [14]:
trainer.test(model, test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9925925731658936
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.9925925731658936}]